# Image Generator

In [ ]:
#| default_exp image_generator

In [ ]:
#| eval:false
from Cloud2DImageConverter import spherical_projection as sp
from Cloud2DImageConverter import data
from matplotlib import pyplot as plt
import numpy as np
import shutil
import time
import os

Carregando os dados

In [ ]:
#| eval:false
point_cloud = data.load_data("../point_clouds/semantic_kitti")

Gerando as projeções* rogerio_dict: Rogério https://github.com/alunos-pfc/Spherical-Projection-SemanticKITTI


In [ ]:
#| eval:false
projection_dict = {
    "reflectance": [],
    "label": []
}

for points in point_cloud:
    rows, cols, intensity, label = sp.spherical_projection_2(points)
    no_label, with_label = sp.create_ImageMatrix(rows, cols, intensity, label)
    with_label = sp.colored_matrix_with_label(with_label)
    projection_dict["reflectance"].append(no_label)
    projection_dict["label"].append(with_label)

In [ ]:
#| eval:false
def mapping_colors(value_dict):
    if value_dict["val"] == "reflectance":
        return value_dict["pixel_values"]
    colors = [color_map[label] for label in value_dict["pixel_values"]]
    return colors
    
color_map = dict(data.color_map)
for key in color_map.keys():
    color_map[key] = [round(val/255, 2) for val in color_map[key]]

In [ ]:
#| eval:false
def rogerio_image_generator(projection_dict, output_folder, rows=64, cols=1024, delete=True):
    start_time = time.time() 
    results_folder =  output_folder+"results"

    # Se a pasta já existir, apague-a 
    if os.path.exists(results_folder) and delete:
        shutil.rmtree(results_folder)
        
    os.makedirs(results_folder)
    os.makedirs(results_folder+"/reflectance")
    os.makedirs(results_folder+"/label")

    for zip_dict in zip(projection_dict["reflectance"], projection_dict["label"]):
        for i, value in enumerate(projection_dict.keys()): 
            if value == "reflectance":
                cmap = "jet"
            else:
                cmap = None

            fig, ax = plt.subplots(figsize=(cols/100,rows/100))
            # Plote a primeira matriz no primeiro subplot
            ax.imshow(zip_dict[i], cmap=cmap)
            ax.xaxis.set_visible(False)    
            ax.yaxis.set_visible(False)  

            # dict["val"] é o valor de reflectance ou label
            folder_type = "/" + value

            # Caminho final para salver
            final_path = os.path.join(results_folder+folder_type)

            # Define o nome do arquivo, tais como 000001, 000002...
            file_number = len(os.listdir(final_path)) + 1
            file_name = f"{final_path}/{file_number:06d}.png"

            plt.savefig(file_name, bbox_inches='tight', pad_inches=0, format='png', dpi=100)
            plt.close()
            
    end_time = time.time()  # Registra o tempo de fim da execução
    elapsed_time = end_time - start_time
    print(f"Tempo de execução: {elapsed_time} segundos")

rogerio_image_generator(projection_dict, output_folder="../point_clouds/")

Tempo de execução: 15.139457941055298 segundos
